In [1]:

### Need to add total innings in phases of the game.

import os, json
import numpy as np, pandas as pd

from pprint import pprint
from datetime import datetime as dt


In [2]:

csv_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_bbb\\'
exp_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\exp\\'
loc_lups = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\lineups\\'


In [3]:

print(csv_location)
print()
print(exp_location)
print()
print(dt.now().date())


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_json_bbb\

C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\exp\

2023-05-05


In [4]:

folder_path = csv_location

# Get a list of all files and directories in the folder
all_files_and_dirs = os.listdir(folder_path)

# Filter the list to include only files (exclude directories)
files_only = [f for f in all_files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

# Print total # of files
print(len(files_only))

# Print names of first few files 
pprint(files_only[:5])


44
['2023_10_Sunrisers Hyderabad vs Lucknow Super Giants.csv',
 '2023_11_Rajasthan Royals vs Delhi Capitals.csv',
 '2023_12_Mumbai Indians vs Chennai Super Kings.csv',
 '2023_13_Gujarat Titans vs Kolkata Knight Riders.csv',
 '2023_14_Punjab Kings vs Sunrisers Hyderabad.csv']


In [5]:

files_list = files_only
total_files = len(files_list)

#print(np.arange(total_files))
print(total_files)


44


In [6]:

#All file numbers > 1359475 are #2023 IPL onwards
#All file numbers > 1304047 are #2022 IPL onwards


In [7]:

# initialize an empty DataFrame to store all the data
bbb_master_df = pd.DataFrame()

# iterate through each file in the directory
for file_name in os.listdir(csv_location):
    if file_name.endswith('.csv'):
        # read the CSV file into a DataFrame
        file_path = os.path.join(csv_location, file_name)
        df_temp = pd.read_csv(file_path)
        
        # append the data to the master DataFrame
        bbb_master_df = bbb_master_df.append(df_temp)

# print the master DataFrame
print(bbb_master_df.shape)


(10796, 20)


In [8]:

print(bbb_master_df.columns.values.tolist())


['inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs']


In [9]:

display(bbb_master_df.head(3))


,inn,bat_tm,batnum,bat,ball,over_num,bow,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,m.grnd,m.date,ss,m.type,m.num,m.vs
0,1,Sunrisers Hyderabad,1,Anmolpreet Singh,0,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants
1,1,Sunrisers Hyderabad,2,MA Agarwal,1,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants
2,1,Sunrisers Hyderabad,1,Anmolpreet Singh,2,0,KR Mayers,1,0,NaN,1,1,0,0,Lucknow,2023-04-07,2023,T20,10,Sunrisers Hyderabad vs Lucknow Super Giants


In [10]:

bbb_master_df.describe()


,inn,batnum,ball,over_num,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,ss,m.num
count,10796.000000,10796.00000,10796.000000,10796.000000,10796.000000,10796.000000,0.0,10796.000000,10796.000000,10796.000000,10796.000000,10796.0,10796.000000
mean,1.489904,3.80289,2.640793,9.379585,1.351426,0.053168,NaN,0.960356,0.305854,0.504446,0.189700,2023.0,22.619303
std,0.499921,2.27426,1.829889,5.729396,1.721797,0.224378,NaN,0.195131,0.460790,0.500003,0.392082,0.0,12.691404
min,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,2023.0,1.000000
25%,1.000000,2.00000,1.000000,4.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,2023.0,12.000000
50%,1.000000,3.00000,3.000000,9.000000,1.000000,0.000000,NaN,1.000000,0.000000,1.000000,0.000000,2023.0,23.000000
75%,2.000000,5.00000,4.000000,14.000000,1.000000,0.000000,NaN,1.000000,1.000000,1.000000,0.000000,2023.0,34.000000
max,2.000000,11.00000,10.000000,19.000000,6.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,2023.0,44.000000


#### Add bow_tm field for use

In [11]:

# Create a dictionary with the match number ('m.num') as the key and the team names as the values
team_mapping = bbb_master_df.groupby('m.num')['bat_tm'].unique().apply(list).to_dict()

# Define a function to switch the team name
def switch_team_name(team, match_num):
    teams = team_mapping[match_num]
    return teams[0] if teams[1] == team else teams[1]

# Add a new column 'bow_team' in the bowling_match_level_df
bbb_master_df = bbb_master_df.copy()
bbb_master_df['bow_tm'] = bbb_master_df.apply(lambda x: switch_team_name(x['bat_tm'], x['m.num']), axis=1)


In [12]:

print(len(bbb_master_df['m.num'].unique()))
#print(pre_master_df['m.vs'].unique())


44



#### Add the bowling and batting flags


In [13]:

print(bbb_master_df.columns.values.tolist())


['inn', 'bat_tm', 'batnum', 'bat', 'ball', 'over_num', 'bow', 'runs', 'wkt', 'wkt_how', 'b.cnt', 'f.ov.powp', 'f.ov.midd', 'f.ov.deat', 'm.grnd', 'm.date', 'ss', 'm.type', 'm.num', 'm.vs', 'bow_tm']


In [14]:

#### Add the linups data to the pstats dataset before more merges

lups = pd.read_excel(loc_lups + 'Tags.xlsx')

print(lups.shape)
print(lups.columns.values.tolist())


(75, 8)
['tm', 'name', 'fname', 'battype', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2']


In [17]:

display(lups.tm.unique())


array(['Kolkata Knight Riders', 'Sunrisers Hyderabad', 'Rajasthan Royals',
       'Gujarat Titans'], dtype=object)

In [ ]:

batcols = ['tm','name','fname','battype']

pre_master_df = pd.merge(lups[batcols], bbb_master_df, how = 'right', right_on = ['bat_tm','bat'], left_on = ['tm','name'])

print(pre_master_df.shape)
print()
print(pre_master_df.columns.values.tolist())


In [ ]:

bowcols = ['tm','name','fname','bowltype','bowlarm','bowlcat','bowlcat.l2']

master_df = pd.merge(pre_master_df \
                                        , lups[bowcols]\
                                        , how = 'left'\
                                        , left_on = ['bow_tm','bow']\
                                        , right_on = ['tm','name']\
                     ,suffixes = ['_bat','_bow'])

print(master_df.shape)
print()
print(master_df.columns.values.tolist())


In [ ]:

print(master_df.bat_tm.unique())


In [ ]:

display( master_df.groupby(['m.num', 'inn','m.vs', 'bat_tm','bow_tm']).agg({'runs':'count'}).reset_index() )


In [ ]:

#master_df.groupby(['over_num']).agg({'runs':'sum'}).reset_index()


In [ ]:

#master_df.groupby(['f.ov.powp','f.ov.midd','f.ov.deat','batnum'], dropna = False).agg({'runs':'sum'}).reset_index()


In [ ]:

#master_df.groupby(['f.ov.powp','f.ov.midd','f.ov.deat','over_num'], dropna = False).agg({'runs':'sum'}).reset_index().sort_values(by=['over_num'])


## Batting stats 

In [ ]:

print(master_df.columns.values.tolist())


In [ ]:

#print(df.shape)


In [ ]:

groupbycols = ['bat_tm', 'batnum', 'bat', 'battype', 'm.num']

#bat_grouped_df = master_df.groupby(['bat_tm', 'batnum', 'bat', 'm.num'])
bat_grouped_df = master_df.groupby(groupbycols) # batnum unique as the m.num per player

def custom_agg(group):
    data = {
        'tr': group['runs'].sum(),
        'tbf': group['b.cnt'].sum(),
        'tbinn': group['m.num'].nunique(),
        'tout': group['wkt'].sum(),
        
        'tr_pp': (group['f.ov.powp'] * group['runs']).sum(),
        'tbf_pp': group['f.ov.powp'].sum(),
        'tout_pp': (group['wkt'] * group['f.ov.powp']).sum(),
        
        'tr_mid': (group['f.ov.midd'] * group['runs']).sum(),
        'tbf_mid': group['f.ov.midd'].sum(),
        'tout_mid': (group['wkt'] * group['f.ov.midd']).sum(),
        
        'tr_dth': (group['f.ov.deat'] * group['runs']).sum(),
        'tbf_dth': group['f.ov.deat'].sum(),
        'tout_dth': (group['wkt'] * group['f.ov.deat']).sum()
    }
    return pd.Series(data)

batting_match_level_df = bat_grouped_df.apply(custom_agg)

# Reset the index so that the primary key columns become regular columns
batting_match_level_df.reset_index(inplace=True)


In [ ]:

#batting_match_level_df.sum()


In [ ]:

print(batting_match_level_df.shape)


In [ ]:

display(batting_match_level_df.head(3))


In [ ]:

# Add points columns to the original DataFrame

batting_match_level_df['f.cp.pts30'] = batting_match_level_df['tr'].apply(lambda x: 4 if x >= 30 else 0)
batting_match_level_df['f.cp.pts50'] = batting_match_level_df['tr'].apply(lambda x: 8 if x >= 50 else 0)
batting_match_level_df['f.cp.pts100'] = batting_match_level_df['tr'].apply(lambda x: 16 if x >= 100 else 0)
batting_match_level_df['f.cp.pts0'] = batting_match_level_df['tr'].apply(lambda x: -2 if x == 0 else 0)


In [ ]:

batting_match_level_df.describe()


In [ ]:

print(batting_match_level_df.shape)
print()
print(batting_match_level_df.columns.values.tolist())


### Batting Transpose

In [ ]:

df_bat_transpose = batting_match_level_df.copy()
df_bat_transpose['game.num'] = 'G' + df_bat_transpose['m.num'].astype(str).str.zfill(2)

df_bat_transpose.sort_values(by = 'm.num', ascending = False)

batting_gwise = df_bat_transpose.pivot(index= ['bat_tm','bat','battype'], columns= ['game.num'], values= ['tr']).reset_index()

# Fixing multi-level columns
#batting_gwise.columns = batting_gwise.columns.map(lambda x: f'{x[1]}.{x[0]}')

# Flatten only multi-level column index
batting_gwise.columns = batting_gwise.columns.map( lambda x: f'{x[0]}.{x[1]}' if len(x) > 1 else x[0] )

# Reset index
batting_gwise = batting_gwise.reset_index(drop = True).rename(columns = {'bat_tm.':'bat_tm','bat.':'bat', 'battype.':'battype'})


In [ ]:

print(batting_gwise.shape)
print()
print(batting_gwise.columns.values.tolist())


In [ ]:

display(batting_gwise)


### Bowling stats

In [ ]:

print(master_df.columns.values.tolist())


In [ ]:

### DICTIONARY CREATION

### ROLLING UP THE DATASET
def custom_bowling_agg(group):
    data = {
        'tbb': group['b.cnt'].sum(),
        'tbrc': group['runs'].sum(),
        'tbwkt': group['wkt'].sum(),
        'tbov': group['over_num'].nunique(),
        
        'tbrc_powp': (group['f.ov.powp'] * group['runs']).sum(),
        'tbrc_mid': (group['f.ov.midd'] * group['runs']).sum(),
        'tbrc_dth': (group['f.ov.deat'] * group['runs']).sum(),
        
        'tbwkt_powp': (group['f.ov.powp'] * group['wkt']).sum(),
        'tbwkt_mid': (group['f.ov.midd'] * group['wkt']).sum(),
        'tbwkt_dth': (group['f.ov.deat'] * group['wkt']).sum()
    }
    return pd.Series(data)

bow_grouped_df = master_df.groupby(['bow_tm', 'bow', 'bowltype', 'bowlarm', 'bowlcat', 'bowlcat.l2', 'm.num'])

bowling_match_level_df_pre = bow_grouped_df.apply(custom_bowling_agg)
bowling_match_level_df_pre.reset_index(inplace=True)

print(master_df.shape)
print(bowling_match_level_df_pre.shape)


In [ ]:

bowling_match_level_df_pre.describe()


In [ ]:

print(master_df.shape)

# master_df for each phase of the game
powp_df = master_df[master_df['f.ov.powp'] != 0]
mid_df = master_df[master_df['f.ov.midd'] != 0]
dth_df = master_df[master_df['f.ov.deat'] != 0]

print(powp_df.shape)
print(mid_df.shape)
print(dth_df.shape)

# Group the DataFrames by the bowler, batting team, and match number
#powp_grouped = powp_df.groupby(['bow_tm', 'bow', 'm.num']).agg(tot_ov_powp = ('over_num','nunique')).reset_index()
powp_grouped = powp_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tbov.powp'})
mid_grouped = mid_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tbov.mid'})
dth_grouped = dth_df.groupby(['bow_tm', 'bow', 'm.num']).agg({'over_num':'nunique'}).reset_index().rename(columns = {'over_num':'tbov.dth'})


In [ ]:

# Reset the index of bowling_match_level_df
#bow_grouped_df.reset_index()

bowling_match_level_df = bowling_match_level_df_pre.reset_index()

# Merge the unique overs bowled in each phase back to the bowling_match_level_df
bowling_match_level_df = bowling_match_level_df.merge(powp_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')
bowling_match_level_df = bowling_match_level_df.merge(mid_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')
bowling_match_level_df = bowling_match_level_df.merge(dth_grouped, on=['bow_tm', 'bow', 'm.num'], how='left')


In [ ]:

print(bowling_match_level_df.columns.values.tolist())


In [ ]:

# BOWLING

bowling_match_level_df['f.cp.pts3w'] = bowling_match_level_df['tbwkt'].apply(lambda x : 4 if x >= 3 else 0 )
bowling_match_level_df['f.cp.pts4w'] = bowling_match_level_df['tbwkt'].apply(lambda x : 8 if x >= 4 else 0 )
bowling_match_level_df['f.cp.pts5w'] = bowling_match_level_df['tbwkt'].apply(lambda x : 16 if x >= 5 else 0 )

bowling_match_level_df['tbecon'] = ( bowling_match_level_df['tbrc'] * 6 ) / bowling_match_level_df['tbb']

bowling_match_level_df['f.cp.ptsecon'] = bowling_match_level_df.apply(lambda x : (2*6)/x.tbb if (5 <= x.tbecon) & ( x.tbecon < 6) \
                                                            else ( (4*6)/x.tbb if (5 <= x.tbecon) & ( x.tbecon < 6) else 0 )\
                                                , axis =1)


In [ ]:

display(bowling_match_level_df.head(3))


In [ ]:

bowling_match_level_df.describe()


In [ ]:

# QC
# bowling_match_level_df[bowling_match_level_df['tov.powp'] == 4]


## Bowling Transpose

In [ ]:

df_bow_transpose = bowling_match_level_df.copy()
df_bow_transpose['game.num'] = 'G' + df_bow_transpose['m.num'].astype(str).str.zfill(2)

df_bow_transpose.sort_values(by = 'game.num', ascending = True)

bowling_gwise = df_bow_transpose.pivot( index= ['bow_tm','bow'], columns= ['game.num'], values= ['tbwkt'] ).reset_index()

# Fixing multi-level columns
#bowling_gwise.columns = bowling_gwise.columns.map(lambda x: f'{x[1]}.{x[0]}')

# Flatten only multi-level column index
bowling_gwise.columns = bowling_gwise.columns.map( lambda x: f'{x[0]}.{x[1]}' if len(x) > 1 else x[0] )

# Reset index
bowling_gwise = bowling_gwise.reset_index(drop = True).rename(columns = {'bow_tm.':'bow_tm','bow.':'bow'})


In [ ]:

print(bowling_gwise.shape)
print()
print(bowling_gwise.columns.values.tolist())


In [ ]:

display(bowling_gwise)


## MERGE

In [ ]:

### MERGE BAT AND BOW DF

# Reset the index for both DataFrames to prepare for merging
batmldf = batting_match_level_df.reset_index(drop = True)
bowmldf = bowling_match_level_df.reset_index(drop = True)

#dropcols = ['bow_tm','bow']
dropcols = []

# Merge the batting figures with the bowling stats
merged_predf = pd.merge( batmldf, bowmldf, 
                     left_on=['bat_tm', 'bat', 'm.num'],
                     right_on=['bow_tm', 'bow', 'm.num'],
                     how='outer', 
                     suffixes=('_bat', '_bowl'))#.rename(columns = {'bat_tm':'tm', 'bat':'name' }).drop(columns = dropcols)

print(merged_predf.shape)
print(merged_predf.columns.values.tolist())

# Replace NaN values with 0 in the merged DataFrame
#merged_predf.fillna(0, inplace=True)



In [ ]:

display(merged_predf.head(5))


In [ ]:

# Function to combine two columns with condition
def combine_columns(row, col1, col2):
    return row[col1] if pd.notna(row[col1]) and row[col1] != '' else row[col2]

# Create a new column using the custom function
merged_predf['tm'] = merged_predf.apply(lambda row: combine_columns(row, 'bat_tm', 'bow_tm'), axis=1)
merged_predf['name'] = merged_predf.apply(lambda row: combine_columns(row, 'bat', 'bow'), axis=1)


In [ ]:

display(merged_predf.tail(5))


In [ ]:

#merged_df.to_csv(exp_location + 'merged.csv', index = False)


### MAIN RUP

In [ ]:


# Merged PreDF has the both the batting and bowling stats for each player and can be easily rolled up to see the results


In [ ]:

#### Rollup at a Player Level

# Replace NaN values with False in the mask
#mask = merged_predf.tm.str.contains(team_for_code).fillna(False)

# Apply the mask to filter the DataFrame
#merged_for_team_only = merged_df[merged_df.tm.str.contains(team_for_code)]
#merged_for_team_only = merged_predf[mask]
merged_for_team_only = merged_predf

tstats = merged_for_team_only.groupby(['tm']).agg({
    #'batnum': np.min,
    'm.num': 'nunique',
    'tr': 'sum',
    'tbf': 'sum',
    #'tbinn': 'sum',
    'tout': 'sum',
    'tr_pp': 'sum',
    'tbf_pp': 'sum',
    'tout_pp': 'sum',
    'tr_mid': 'sum',
    'tbf_mid': 'sum',
    'tout_mid': 'sum',
    'tr_dth': 'sum',
    'tbf_dth': 'sum',
    'tout_dth': 'sum',
    #'f.cp.pts30': 'sum',
    #'f.cp.pts50': 'sum',
    #'f.cp.pts100': 'sum',
    #'f.cp.pts0': 'sum',    
    'tbb': 'sum',
    'tbrc': 'sum',
    'tbwkt': 'sum',
    'tbov': 'sum',
    'tbov.powp':'sum', 
    'tbov.mid':'sum', 
    'tbov.dth':'sum',
    'tbrc_powp': 'sum',
    'tbrc_mid': 'sum',
    'tbrc_dth': 'sum',
    'tbwkt_powp': 'sum',
    'tbwkt_mid': 'sum',
    'tbwkt_dth': 'sum'
}).reset_index().rename(columns = {'batnum' : 'batnum_min','m.num':'tinn'})

print(tstats.shape)


In [ ]:

print(tstats.columns.values.tolist())


In [ ]:

display(tstats.transpose())


In [ ]:
# Now check the gwise battin and bowling stats to append to pstats

## Percentage split

In [ ]:

pct_splits = tstats.copy()


In [ ]:

pct_splits['tr%.pp'] = round(100*pct_splits['tr_pp'] / ( pct_splits['tr'] ),0)
pct_splits['tr%.mid'] = round(100*pct_splits['tr_mid'] / ( pct_splits['tr'] ),0)
pct_splits['tr%.dth'] = round(100*pct_splits['tr_dth'] / ( pct_splits['tr'] ),0)


In [ ]:

pct_splits['tbov%.pp'] = round(100*pct_splits['tbov.powp'] / ( pct_splits['tbov'] ),0)
pct_splits['tbov%.mid'] = round(100*pct_splits['tbov.mid'] / ( pct_splits['tbov'] ),0)
pct_splits['tbov%.dth'] = round(100*pct_splits['tbov.dth'] / ( pct_splits['tbov'] ),0)


In [ ]:

pct_splits['tbwkt%.pp'] = round(100*pct_splits['tbwkt_powp'] / ( pct_splits['tbwkt'] ),0)
pct_splits['tbwkt%.mid'] = round(100*pct_splits['tbwkt_mid'] / ( pct_splits['tbwkt'] ),0)
pct_splits['tbwkt%.dth'] = round(100*pct_splits['tbwkt_dth'] / ( pct_splits['tbwkt'] ),0)


In [ ]:

print(pct_splits.shape)
print()
print(pct_splits.columns.values.tolist())
print()


In [ ]:


pdetails = ['name','bowltype','bowlarm','bowlcat','bowlcat.l2']

tr_phs = ['tr_pp','tr_mid','tr_dth']
tbf_phs = ['tbf_pp','tbf_mid','tbf_dth']
tout_phs = ['tout_pp','tout_mid','tout_dth']

tbov_phs = ['tbov.powp','tbov.mid','tbov.dth']
tbwkt_phs = ['tbwkt_powp','tbwkt_mid','tbwkt_dth']
tbrc_phs = ['tbrc_powp','tbrc_mid','tbrc_dth']

tbov_pct = ['tbov%.pp', 'tbov%.mid', 'tbov%.dth']

#f_cp = ['f.cp.pts30', 'f.cp.pts50', 'f.cp.pts100', 'f.cp.pts0']

#misc = ['bat_tm','bow_tm']


In [ ]:

dfexp = pct_splits.drop(columns = ['tbf'] + tbf_phs + ['tbov'] + tbov_phs + tbov_pct)
#dfexp = pct_splits.copy()


In [ ]:

print(dfexp.columns.values.tolist())


In [ ]:

# Change the column order
new_order = ['tm', 'tinn', 'tr'\
             , 'tr_pp', 'tr_mid', 'tr_dth'\
             , 'tout_pp', 'tout_mid', 'tout_dth'\
             , 'tbb', 'tbrc', 'tbrc_powp', 'tbrc_mid', 'tbrc_dth'\
             , 'tbwkt', 'tbwkt_powp', 'tbwkt_mid', 'tbwkt_dth'\
             , 'tr%.pp', 'tr%.mid', 'tr%.dth'\
             , 'tbwkt%.pp', 'tbwkt%.mid', 'tbwkt%.dth']
expfile = dfexp[new_order].copy()


In [ ]:

exp_file = f'tstats_all.xlsx'
dfexp.to_excel(exp_location + exp_file, index = False)


### Fantasy Points Totals

In [ ]:
'''
#Create Fantasy pts crickpe version

pstats['bat_pts'] = master_df['rs'].fillna(0) + master_df['four'].fillna(0) + master_df['six'].fillna(0)*2 \
    + master_df['pts_for_thrty'].fillna(0) + master_df['pts_for_fifty'].fillna(0) + master_df['pts_for_hundy'].fillna(0) + master_df['pts_for_ducky'].fillna(0)

master_df['bow_pts'] = master_df['wkt'].fillna(0)*25 \
    + master_df['pts_for_3w'].fillna(0) + master_df['pts_for_4w'].fillna(0) + master_df['pts_for_5w'].fillna(0) \
        + master_df['pts_for_bow_econ'].fillna(0)

master_df['pts_crickpe'] = master_df['bat_pts'].fillna(0) + master_df['bow_pts'].fillna(0)
'''

In [ ]:

# Calculate strike rates for each phase of the game
#df_plyr_mtch['sr_powp'] = (df_plyr_mtch['tot_runs_powp'] / df_plyr_mtch['tot_balls_powp']) * 100
#df_plyr_mtch['sr_mid'] = (df_plyr_mtch['tot_runs_mid'] / df_plyr_mtch['tot_balls_mid']) * 100
#df_plyr_mtch['sr_dth'] = (df_plyr_mtch['tot_runs_dth'] / df_plyr_mtch['tot_balls_dth']) * 100
